In [1]:
pip install -e ..

Obtaining file:///home/jovyan/work/Phase5/Assessments/DataJobs
  Attempting uninstall: src
    Found existing installation: src 0.1.0
    Uninstalling src-0.1.0:
      Successfully uninstalled src-0.1.0
  Running setup.py develop for src
Note: you may need to restart the kernel to use updated packages.


In [2]:
from definitions import ROOT_DIR
print(ROOT_DIR)

/home/jovyan/work/Phase5/Assessments/DataJobs


In [3]:
from importlib import reload
import src.data.mytokenizer
reload(src.data.mytokenizer)

<module 'src.data.mytokenizer' from '/home/jovyan/work/Phase5/Assessments/DataJobs/src/data/mytokenizer.py'>

In [4]:
from src.data import job_database

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.pipeline import Pipeline
import inspect
from IPython.display import Markdown, display
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

def display_object(obj):
    """Summary of display_object. Get source code for the provided object 
    and display in notebook as markdown.
    """
    source = inspect.getsource(obj)
    wrapped_source = f'```python\n{source}\n```'
    markdown_source = Markdown(wrapped_source)
    display(markdown_source)

In [6]:
query = """
SELECT * from jobs
WHERE title LIKE '%data%'
OR description LIKE '%data%';
"""
db = job_database.JobsDb(ROOT_DIR+'/data/external/jobs.sqlite')
data = db.load_query_as_df(query)
db.close()

In [7]:
descriptions = data['description'].tail(100)

In [8]:
doc = descriptions[19074]

In [9]:
tokenizer = src.data.mytokenizer.LemmaTokenizer()

In [10]:
tokenizer(doc)

['this',
 'position',
 'work',
 'in',
 'a',
 'warehouse',
 'setting',
 'the',
 'data',
 'entryqc',
 'professional',
 'must',
 'periodically',
 'collect',
 'and',
 'review',
 'report',
 'to',
 'uncover',
 'any',
 'data',
 'error',
 'and',
 'to',
 'coordinate',
 'correction',
 'with',
 'routine',
 'use',
 'of',
 'standard',
 'office',
 'equipment',
 'such',
 'as',
 'dual',
 'computer',
 'scanner',
 'and',
 'file',
 'etc',
 'this',
 'be',
 'a',
 'multistep',
 'hiring',
 'process',
 'applicant',
 'must',
 'be',
 'flexible',
 'and',
 'available',
 'for',
 'a',
 'phone',
 'screen',
 'if',
 'the',
 'client',
 'be',
 'interested',
 'must',
 'be',
 'available',
 'to',
 'work',
 'monday',
 'sunday',
 'as',
 'need',
 'this',
 'be',
 'not',
 'a',
 'remote',
 'position',
 'shift',
 'available',
 '800am',
 '400pm',
 '400pm',
 '1200am',
 'responsibility',
 'maintains',
 'database',
 'by',
 'enter',
 'new',
 'and',
 'updated',
 'customer',
 'and',
 'account',
 'information',
 'prepares',
 'source',
 '

In [33]:
pipe = Pipeline(
    steps = [
        ('vectorizer', CountVectorizer(
            encoding='utf-8',
            decode_error='ignore',
            strip_accents='unicode',
            lowercase=True,
            preprocessor=None,
            tokenizer=src.data.mytokenizer.LemmaTokenizer(),
            stop_words='english',
            ngram_range=(1,4),
            analyzer='word',
            max_df=0.95,
            min_df=0.05,
            max_features=None
                                      )),
        ('topicModel', LatentDirichletAllocation())
    ]
)

In [34]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [35]:
%time pipe.fit(descriptions)

CPU times: user 8.47 s, sys: 71.9 ms, total: 8.54 s
Wall time: 8.55 s


Pipeline(steps=[('vectorizer',
                 CountVectorizer(decode_error='ignore', max_df=0.95,
                                 min_df=0.05, ngram_range=(1, 4),
                                 stop_words='english', strip_accents='unicode',
                                 tokenizer=<src.data.mytokenizer.LemmaTokenizer object at 0x7fb56a990e20>)),
                ('topicModel', LatentDirichletAllocation())])

In [36]:
%time pipe.transform(descriptions)

CPU times: user 7.19 s, sys: 15.9 ms, total: 7.21 s
Wall time: 7.21 s


array([[5.61940588e-04, 6.73475390e-02, 3.06053524e-01, 5.61898237e-04,
        5.61868265e-04, 5.61900202e-04, 5.61830132e-04, 9.45444060e-02,
        3.86223724e-01, 1.43021370e-01],
       [6.75779187e-04, 6.75901574e-04, 6.75892148e-04, 6.75828573e-04,
        6.75826965e-04, 6.75776891e-04, 6.75728541e-04, 6.75873356e-04,
        9.53464615e-01, 4.11287776e-02],
       [1.44016180e-01, 3.84712520e-04, 2.23509475e-01, 1.31966405e-01,
        3.84686891e-04, 3.84703121e-04, 3.84670912e-04, 3.84691826e-04,
        4.98199782e-01, 3.84692457e-04],
       [4.20244830e-04, 4.20254505e-04, 4.20240734e-04, 4.20250471e-04,
        9.96217830e-01, 4.20235356e-04, 4.20197718e-04, 4.20241378e-04,
        4.20261154e-04, 4.20243580e-04],
       [1.49270906e-03, 1.49268980e-03, 7.98975401e-01, 1.49284574e-03,
        1.49280054e-03, 1.49287093e-03, 1.49279251e-03, 1.49281986e-03,
        1.89082348e-01, 1.49272273e-03],
       [3.71032102e-01, 1.59768982e-04, 1.72342888e-01, 1.59795730e-04,
   

In [37]:
tokens = pipe.named_steps['vectorizer'].get_feature_names()

In [38]:
len(tokens)

1548

In [39]:
tokens

['1',
 '10',
 '10 year',
 '100',
 '12',
 '15',
 '2',
 '2 year',
 '2 year experience',
 '2020',
 '3',
 '3 year',
 '3 year experience',
 '4',
 '40',
 '401',
 '401 k',
 '5',
 '5 year',
 '5 year experience',
 '6',
 '7',
 'ability',
 'ability communicate',
 'ability manage',
 'ability work',
 'ability work effectively',
 'ability work independently',
 'able',
 'accept',
 'access',
 'accommodation',
 'accord',
 'account',
 'accountability',
 'accuracy',
 'accurate',
 'achieve',
 'acquisition',
 'act',
 'action',
 'action employer',
 'action employer qualify',
 'action employer qualify applicant',
 'actionable',
 'active',
 'activity',
 'acumen',
 'addition',
 'additional',
 'address',
 'administer',
 'advance',
 'advance diversity',
 'advanced',
 'affirmative',
 'affirmative action',
 'age',
 'age marital',
 'age marital status',
 'age national',
 'age national origin',
 'agency',
 'agile',
 'ai',
 'aid',
 'air',
 'airflow',
 'algorithm',
 'algorithms',
 'align',
 'alignment',
 'allow',
 'an